# Part One

In [13]:
import matplotlib.path as mpath
from collections import Counter
from math import floor

In [10]:
def getPuzzleInput():
	with open("Day 9.txt") as file:
		return file.read()

def getArea(point1, point2):
	return (abs(point1[0] - point2[0]) + 1) * (abs(point1[1] - point2[1]) + 1)
	

In [11]:
points = [[int(x.split(",")[0]), int(x.split(",")[1])] for x in getPuzzleInput().split("\n")]
pointsCombinations = []

for i in range(len(points)):
	point1 = points[i]
	for j in range(len(points)):
		if i == j:
			continue
		point2 = points[j]
		pointsCombinations.append((point1, point2, getArea(point1, point2)))
pointsCombinations.sort(key=lambda x: x[2], reverse=True)

print(pointsCombinations[0][2])

4767418746


In [12]:
# one liner for fun
print(max([getArea(points[i], points[j]) for i in range(len(getPuzzleInput().split('\n'))) for j in range(len(getPuzzleInput().split('\n'))) if i != j]))

4767418746


# Part Two

In [14]:
points = [(int(x.split(",")[0]), int(x.split(",")[1])) for x in getPuzzleInput().split("\n")]

def getBorder(points: list[list], type:str="inner"):
	borderPoints = []
	for i in range(len(points)): # so it wraps
		x1, y1 = list(points[i%len(points)])
		x2, y2 = list(points[(i+1)%len(points)])

		e = []

		# going down
		if x1 < x2:
			if type == "outer":
				e = [(x,y) for x in range(x1-1, x2+2) for y in range(y1-1, y2)]
			else:
				e = [(x,y) for x in range(x1, x2+1) for y in range(y1, y2+1)]
		
		# going right
		elif y1 < y2:
			if type == "outer":
				e = [(x,y) for x in range(x1+1, x2+2) for y in range(y1-1, y2+2)]
			else:
				e = [(x,y) for x in range(x1, x2+1) for y in range(y1, y2+1)]

		# going up
		elif x1 > x2:
			if type == "outer":
				e = [(x,y) for x in range(x2-1, x1+2) for y in range(y1+1, y2+2)]
			else:
				e = [(x,y) for x in range(x2, x1+1) for y in range(y1, y2+1)]

		# going left
		elif y1 > y2:
			if type == "outer":
				e = [(x,y) for x in range(x1-1, x2) for y in range(y2-1, y1+2)]
			else:
				e = [(x,y) for x in range(x1, x2+1) for y in range(y2, y1+1)]

		borderPoints.extend(e)
	
	return borderPoints

def getCorners(borderPoints):
	count = Counter(borderPoints)
	c = [b for b in borderPoints if count[b] > 1]
	return list(dict.fromkeys(c))

outerPath = mpath.Path(getCorners(getBorder(points, "outer")))

# Go through all of the possible combinations of points around the shape
# Manually changed the length of the loop because I know where
# the largest rectangle should show up based on the shape of the border
for i in range(floor(len(pointsCombinations)*.4), floor(len(pointsCombinations)*.8)):
	# print("hi")
	pc1, pc2 = pointsCombinations[i][0:2]
	
	rectPoints = getBorder([pc1, [pc1[0], pc2[1]], pc2, [pc2[0], pc1[1]]])
	# print(rectPoints)
	
	rectPath = mpath.Path(rectPoints)

	# If the points of the rectangle is completely contained within the path the corners make
	if outerPath.contains_points(rectPoints).all():
		# since the list of point combinations is sorted by greatest area, 
		# once we find the largest one that fits we can return
		print(pc1, pc2)
		print(getArea(pc1, pc2))
		break

[5247, 66451] [94697, 50108]
1461987144
